In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prepare a dataset

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset, DatasetDict

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/NLP_data/gpt_60_emotions.csv")

df = dataset.dropna()

# Define encoding dictionary
emotion_encoding = {'anger': 0, 'disgust': 1, 'fear': 2, 'happiness': 3, 'sadness': 4, 'surprise': 5}

# Map emotions to integers
df['emotion'] = df['emotion'].map(emotion_encoding)


df

,sentence,emotion
0,I can't believe they cancelled my flight witho...,0
1,It's infuriating when people don't listen to s...,0
2,How dare they blame me for their own mistake!,0
3,The level of incompetence in this office is ju...,0
4,"Every time I see the mess, it just makes my bl...",0
...,...,...
361,The unexpected compliment brightened my day si...,5
362,I was taken aback by the depth of her insight.,5
363,The sudden downpour during our picnic was a su...,5
364,The unexpected twist in the movie left everyon...,5


In [ ]:
data_train, data_test = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(data_train.rename(columns={"sentence": "text", "emotion": "label"}))
test_dataset = Dataset.from_pandas(data_test.rename(columns={"sentence": "text", "emotion": "label"}))

# Create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
"""from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]"""

'from datasets import load_dataset\n\ndataset = load_dataset("yelp_review_full")\ndataset["train"][100]'

As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) method to apply a preprocessing function over the entire dataset:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

Map:   0%|          | 0/292 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(292))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(74))

In [ ]:
small_train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 292
})

<a id='trainer'></a>

## Train

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

<Tip>

You will see a warning about some of the pretrained weights not being used and some weights being randomly
initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

</Tip>

### Training hyperparameters

Next, create a [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments), but feel free to experiment with these to find your optimal settings.

Specify where to save the checkpoints from your training:

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer")

### Evaluate

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) does not automatically evaluate model performance during training. You'll need to pass [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) a function to compute and report metrics. The [🤗 Evaluate](https://huggingface.co/docs/evaluate/index) library provides a simple [`accuracy`](https://huggingface.co/spaces/evaluate-metric/accuracy) function you can load with the [evaluate.load](https://huggingface.co/docs/evaluate/main/en/package_reference/loading_methods#evaluate.load) (see this [quicktour](https://huggingface.co/docs/evaluate/a_quick_tour) for more information) function:

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 292
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 111


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.979086,0.716216
2,No log,0.443084,0.891892
3,No log,0.332821,0.905405


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 74
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 74
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can s

TrainOutput(global_step=111, training_loss=0.7199931789088894, metrics={'train_runtime': 5931.3228, 'train_samples_per_second': 0.148, 'train_steps_per_second': 0.019, 'total_flos': 230493562232832.0, 'train_loss': 0.7199931789088894, 'epoch': 3.0})

<a id='keras'></a>

In [ ]:
# Save the model
trainer.save_model("my_emotion_model")


Saving model checkpoint to my_emotion_model
Configuration saved in my_emotion_model/config.json
Model weights saved in my_emotion_model/pytorch_model.bin


In [ ]:
# Assuming you also have the tokenizer, save it
tokenizer.save_pretrained("my_emotion_model")

tokenizer config file saved in my_emotion_model/tokenizer_config.json
Special tokens file saved in my_emotion_model/special_tokens_map.json


('my_emotion_model/tokenizer_config.json',
 'my_emotion_model/special_tokens_map.json',
 'my_emotion_model/vocab.txt',
 'my_emotion_model/added_tokens.json',
 'my_emotion_model/tokenizer.json')

## Applying predictions to Kaggle data

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the model and tokenizer
model_path = "/content/drive/MyDrive/NLP_data/fine_tuned_BERT_1"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# Emotion encoding
emotion_encoding = {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'happiness', 4: 'sadness', 5: 'surprise'}

# Function to predict the emotion of a sentence
def predict_emotion(sentence):
    # Encode the sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()

    # Map the predicted class ID to its label
    predicted_emotion = emotion_encoding[predicted_class_id]

    return predicted_emotion


In [ ]:
# Example usage
sentence = "I'm feeling very happy today!"
predicted_emotion = predict_emotion(sentence)
print(f"Predicted emotion: {predicted_emotion}")

In [ ]:
kaggle_base = pd.read_csv('/content/drive/MyDrive/NLP_data/kaggle_data.csv', sep='\t')

In [ ]:
# Prediction function
emotion_encoding = {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'happiness', 4: 'sadness', 5: 'surprise'}
def predict_emotion(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return emotion_encoding[predicted_class_id]


In [ ]:
# Assuming `df` is your DataFrame
predictions = [predict_emotion(sentence) for sentence in kaggle_base['sentence']]

In [ ]:
kaggle_base['predicted_emotion'] = predictions

In [ ]:
# Creating new dataframe without the sentence column:
kaggle_submission = kaggle_base.drop('sentence', axis=1)

kaggle_submission.to_csv('submissions/submission_Transformer_BERT_3.csv', index=False)